In [1]:
import atd2022
import torch
import pandas as pd
import numpy as np
from atd_informer.atd_informer_V2 import ATD_Informer_V2
from utils.tools import dotdict
from atd_wrapper_V2 import InformerForcaster_V2
pd.set_option('display.max_columns', None)
from sktime.performance_metrics.forecasting import (
    MeanAbsoluteScaledError,
    MeanSquaredError,
)

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
torch.cuda.set_device(1) 

# Testing ATD Protocol

In [4]:
truth = atd2022.io.read_csv()

In [9]:
args = dotdict()
args.enc_in = 5200 # encoder input size
args.dec_in = 5200 # decoder input size
args.c_out = 5200 # output size
args.factor = 6 # probsparse attn factor
args.d_model = 1500 # dimension of model
args.n_heads = 18 # num of heads
args.e_layers = 6 # num of encoder layers
args.d_layers = 6 # num of decoder layers
args.d_ff = 3000 # dimension of fcn in model
args.dropout = 0.005 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = True # whether to output attention in ecoder
args.mix = True
args.padding = 0
args.freq = 'w'
args.inverse=False
args.timeenc=1


args.use_gpu=True

#args.cols=1
args.checkpoints = "/scratch/wzong/atd_tmp"



args.seq_len=90
args.label_len=25
args.pred_len=25


args.batch_size = 32
args.learning_rate = 0.001
args.loss = 'mse'
args.lradj = 'type2'
args.use_amp = False

args.itr=1
args.train_epochs=20
args.patience=5

In [10]:
exp = InformerForcaster_V2(args)

In [11]:
exp.fit(truth)

Use GPU: cuda:None
Use GPU: cuda:None
>>>>>>>start training : None_None_ftNone_sl90_ll25_pl25_dm1500_nh18_el6_dl6_df3000_atprob_fc6_ebtimeF_dtTrue_mxTrue_None_0>>>>>>>>>>>>>>>>>>>>>>>>>>
Epoch: 1 cost time: 0.925755500793457
Epoch: 1, Steps: 2 | Train Loss: 4893475.5000000
Validation loss decreased (inf --> 4893475.500000).  Saving model ...
Epoch: 2 cost time: 0.8893454074859619
Epoch: 2, Steps: 2 | Train Loss: 4891720.5000000
Validation loss decreased (4893475.500000 --> 4891720.500000).  Saving model ...
Epoch: 3 cost time: 0.9147562980651855
Epoch: 3, Steps: 2 | Train Loss: 4890465.5000000
Validation loss decreased (4891720.500000 --> 4890465.500000).  Saving model ...
Epoch: 4 cost time: 0.9834203720092773
Epoch: 4, Steps: 2 | Train Loss: 4889153.0000000
Validation loss decreased (4890465.500000 --> 4889153.000000).  Saving model ...
Epoch: 5 cost time: 1.0104670524597168
Epoch: 5, Steps: 2 | Train Loss: 4887764.5000000
Validation loss decreased (4889153.000000 --> 4887764.500000)

In [12]:
test=exp.predict([1,2,3,4])

[[11.075557   4.925238   9.84288   ... 58.52229   59.10325    1.252343 ]
 [11.075587   4.9252944  9.842885  ... 58.522232  59.103256   1.2522944]
 [11.075691   4.9253592  9.842965  ... 58.522186  59.10328    1.2523042]
 ...
 [11.075613   4.9252825  9.8429575 ... 58.522236  59.103264   1.2524289]
 [11.075799   4.9254265  9.84302   ... 58.5222    59.103157   1.2522799]
 [11.075542   4.9253078  9.842912  ... 58.522182  59.10333    1.2523167]]
<class 'numpy.ndarray'>
(25, 5200)


# Full Scaled Testing

In [12]:
args = dotdict()


args.enc_in = 5200 # encoder input size
args.dec_in = 5200 # decoder input size
args.c_out = 5200 # output size
args.factor = 6 # probsparse attn factor
args.d_model = 4096 # dimension of model
args.n_heads = 8 # num of heads
args.e_layers = 5 # num of encoder layers
args.d_layers = 5 # num of decoder layers
args.d_ff = 4096 # dimension of fcn in model
args.dropout = 0.005 # dropout
args.attn = 'prob' # attention used in encoder, options:[prob, full]
args.embed = 'timeF' # time features encoding, options:[timeF, fixed, learned]
args.activation = 'gelu' # activation
args.distil = True # whether to use distilling in encoder
args.output_attention = True # whether to output attention in ecoder
args.mix = True
args.padding = 0
args.freq = 'w'
args.inverse=False
args.timeenc=1


args.use_gpu=True

#args.cols=1
args.checkpoints = "/scratch/wzong/atd_tmp"



args.seq_len=60
args.label_len=20
args.pred_len=1


args.batch_size = 60
args.learning_rate = 0.001
args.loss = 'mse'
args.lradj = 'type2'
args.use_amp = False

args.itr=1
args.train_epochs=2
args.patience=5

In [13]:
lispDL = InformerForcaster_V2(args)

In [14]:
 # Load data
truth = atd2022.io.read_csv()

# Subset the data for the sake of making this fast.
# Remove if you want to run the example on the entire dataset.
truth = truth.head(108)

# Experiment Parameters
window = 100
num_predict = 4
gap = 0
slide = 1

# Create a dataset `Splitter` object for generating train/test splits
splitter = atd2022.backtest.Splitter(
    truth,
    window,
    num_predict,
    gap,
    slide,
    expanding=True,
) 

In [ ]:
 # Populate a list of models that support the `atd2022.forecasters.Forecaster` protocol
# with which will we generate historical forecasts
models = [
    lispDL,
    atd2022.forecasters.PredictMeanForecaster(),
    atd2022.forecasters.ExponentiallyWeightedMovingAverage(),
]

# Compute historical forecasts for all models
predictions = [
    atd2022.backtest.historical_forecast(model, splitter, verbose=True)
    for model in models
]